In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%204%20-%20S%2BP/S%2BP%20Week%203%20Lesson%202%20-%20RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import io
from pathlib import Path

print(tf.__version__)

2.9.0


In [3]:
def rm_tree(pth):
    pth = Path(pth)
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()


if (logdir := Path.joinpath(Path.cwd(), Path("logs"))).exists():
    rm_tree(str(logdir.resolve()))

Path.mkdir(logdir, parents=True, exist_ok=True)

# Clear out prior logging data.
if (plotdir := Path.joinpath(logdir, "plots")).exists():
    rm_tree(str(plotdir.resolve()))

plotdir = Path.joinpath(plotdir, f"{datetime.now().strftime('%Y%m%d-%H%M%S')}")
file_writer = tf.summary.create_file_writer(str(plotdir.resolve()))


def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_series(
    time, series, format="-", start=0, end=None, xlabel="", ylabel="", series_name=""
):
    figure = plt.figure(figsize=(10, 10))
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    with file_writer.as_default():
        tf.summary.image(series_name, plot_to_image(figure), step=0)


def trend(time, slope=0):
    return slope * time


def seasonal_pattern(season_time):
    return np.where(
        season_time < 0.4, np.cos(season_time * 2 * np.pi), 1 / np.exp(3 * season_time)
    )


def seaonality(time, period, amplitude=1, phase=0):
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)


def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [4]:
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

series = (
    baseline + trend(time, slope) + seaonality(time, period=365, amplitude=amplitude)
)

series += noise(time, noise_level, seed=42)
plot_series(time, series, xlabel="Time", ylabel="Value", series_name="series_vs_time")

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]

time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [5]:
def windowed_dataset(
    series,
    window_size,
    batch_size=32,
    shuffle_buffer=None,
    window_shift=1,
):
    def create_target(target_window):
        X = target_window[:-1]
        Y = target_window[-1:]
        return X, Y

    def batch_window(batched_window, size):
        return batched_window.batch(size)

    ds = tf.data.Dataset.from_tensor_slices(series)
    length_dataset = (
        shuffle_buffer
        if shuffle_buffer
        else ds.reduce(0, lambda x, _: x + 1).numpy() * 50 // 100
    )
    ds = ds.window(window_size + 1, shift=window_shift, drop_remainder=True)
    ds = ds.flat_map(lambda window: batch_window(window, window_size + 1))

    ds = ds.shuffle(length_dataset)
    ds = ds.map(create_target, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(batch_size, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.prefetch(1)
    for X_batch, Y_batch in ds.take(1):
        print(X_batch.shape, Y_batch.shape)
    return ds

In [6]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
model = tf.keras.Sequential(
    [
        tf.keras.layers.Lambda(
            lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]
        ),
        tf.keras.layers.SimpleRNN(40, return_sequences=True),
        tf.keras.layers.SimpleRNN(40),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 100.0),
    ]
)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10 ** (epoch / 20)
)
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

(32, 20) (32, 1)
Epoch 1/100


d:\codebase\.venv\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


31/31 [==============================] - 7s 119ms/step - loss: 194.1675 - mae: 194.6675 - lr: 1.0000e-08
Epoch 2/100
31/31 [==============================] - 2s 70ms/step - loss: 188.7974 - mae: 189.2974 - lr: 1.1220e-08
Epoch 3/100
31/31 [==============================] - 2s 51ms/step - loss: 182.0620 - mae: 182.5620 - lr: 1.2589e-08
Epoch 4/100
31/31 [==============================] - 2s 54ms/step - loss: 174.1209 - mae: 174.6209 - lr: 1.4125e-08
Epoch 5/100
31/31 [==============================] - 2s 56ms/step - loss: 164.6153 - mae: 165.1153 - lr: 1.5849e-08
Epoch 6/100
31/31 [==============================] - 2s 60ms/step - loss: 152.8394 - mae: 153.3394 - lr: 1.7783e-08
Epoch 7/100
31/31 [==============================] - 2s 59ms/step - loss: 137.5140 - mae: 138.0140 - lr: 1.9953e-08
Epoch 8/100
31/31 [==============================] - 2s 59ms/step - loss: 116.4385 - mae: 116.9385 - lr: 2.2387e-08
Epoch 9/100
31/31 [==============================] - 2s 49ms/step - loss: 87.4562 -

In [7]:
figure = plt.figure(figsize=(10, 10))
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 30])
with file_writer.as_default():
    tf.summary.image("learning_rate_descent", plot_to_image(figure), step=0)

In [8]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

dataset = windowed_dataset(
    x_train, window_size, batch_size=128, shuffle_buffer=shuffle_buffer_size
)

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Lambda(
            lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]
        ),
        tf.keras.layers.SimpleRNN(40, return_sequences=True),
        tf.keras.layers.SimpleRNN(40),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 100.0),
    ]
)

optimizer = tf.keras.optimizers.SGD(lr=5e-5, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
history = model.fit(dataset, epochs=400)

(128, 20) (128, 1)
Epoch 1/400
8/8 [==============================] - 4s 150ms/step - loss: 83.2483 - mae: 83.7483
Epoch 2/400
8/8 [==============================] - 1s 83ms/step - loss: 26.8753 - mae: 27.3740
Epoch 3/400
8/8 [==============================] - 0s 37ms/step - loss: 19.1454 - mae: 19.6416
Epoch 4/400
8/8 [==============================] - 0s 40ms/step - loss: 16.2221 - mae: 16.7180
Epoch 5/400
8/8 [==============================] - 0s 39ms/step - loss: 12.5505 - mae: 13.0427
Epoch 6/400
8/8 [==============================] - 0s 39ms/step - loss: 9.1453 - mae: 9.6350
Epoch 7/400
8/8 [==============================] - 0s 45ms/step - loss: 12.5072 - mae: 13.0003
Epoch 8/400
8/8 [==============================] - 0s 38ms/step - loss: 8.8993 - mae: 9.3865
Epoch 9/400
8/8 [==============================] - 0s 47ms/step - loss: 8.4742 - mae: 8.9617
Epoch 10/400
8/8 [==============================] - 0s 48ms/step - loss: 8.1046 - mae: 8.5922
Epoch 11/400
8/8 [===================

In [9]:
predict_dataset = (
    tf.data.Dataset.from_tensor_slices(series)
    .window(size=window_size, shift=1, drop_remainder=True)
    .flat_map(lambda window_to_flatten: window_to_flatten.batch(window_size))
    .map(lambda x: tf.expand_dims(x, axis=0))
)

In [10]:
predict_forecast = model.predict(predict_dataset)

1442/1442 [==============================] - 20s 13ms/step


In [11]:
predict_forecast = predict_forecast[split_time - window_size : -1]
predict_results = np.array(predict_forecast)


figure = plt.figure(figsize=(10, 10))
plt.plot(time_valid, x_valid)
plt.plot(time_valid, predict_results)
plt.xlabel("Time")
plt.ylabel("Series")
plt.grid(True)
with file_writer.as_default():
    tf.summary.image("predicted_results_vs_valid_data", plot_to_image(figure), step=0)

# plot_series(time= time_valid, series1=x_valid, series2=results, series_name="forecast_results_with_datset")

In [12]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# -----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
# -----------------------------------------------------------
mae = history.history["mae"]
loss = history.history["loss"]
epochs = range(len(loss))  # Get number of epochs

# ------------------------------------------------
# Plot MAE and Loss
# ------------------------------------------------
figure = plt.figure(figsize=(10, 10))
plt.plot(epochs, mae, "r")
plt.plot(epochs, loss, "b")
plt.title("MAE and Loss")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])
with file_writer.as_default():
    tf.summary.image("mae_loss_vs_epochs", plot_to_image(figure), step=0)


epochs_zoom = epochs[200:]
mae_zoom = mae[200:]
loss_zoom = loss[200:]

# ------------------------------------------------
# Plot Zoomed MAE and Loss
# ------------------------------------------------
figure = plt.figure(figsize=(10, 10))
plt.plot(epochs, mae, "r")
plt.plot(epochs, loss, "b")
plt.title("MAE and Loss")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])
with file_writer.as_default():
    tf.summary.image("mae_loss_vs_epochs_zoomed", plot_to_image(figure), step=0)